# Data Preparation for GeSAI - AB Data Challenge 

In [ ]:
# Import necessary libraries
import pandas as pd

# 1. Official Data. Preparación inicial del dataset AB3  

Dataset information data_ab3: Períodes de fuites detectades (tant per l’operadora com pel client), data de
requeriment aixecat, classificació de la incidència i comunicació amb el client (vía i missatge).

## 1.1. Información básica de nuestro dataset

In [ ]:
# Load dataset from data/official-data/data_ab3.parquet
df_ab3 = pd.read_parquet('../data/official-data/data_ab3_complete.parquet')

In [ ]:
# Display the first few rows of the dataframe
display(df_ab3.head())

,POLISSA_SUBM,DATA_INI_FACT,DATA_FIN_FACT,CREATED_MENSAJE,CODIGO_MENSAJE,TIPO_MENSAJE,US_AIGUA_SUBM,SECCIO_CENSAL,NUMEROSERIECONTADOR,CONSUMO_REAL,FECHA_HORA
0,RGYFWIZ4ZRRZKX2K,2023-09-13 00:00:00,2023-11-14 00:00:00,NaT,None,None,DOMÈSTIC,0801907090,IBAJ44VHSIRRTASA,NaN,2024-01-01
1,HHB4U5HUQKW7IOGD,2023-08-13 00:00:00,2023-10-16 00:00:00,NaT,None,None,DOMÈSTIC,0801909040,L2CLPPJRIPAEESV7,NaN,2024-01-01
2,EU6AT3IKPUKCZTBU,2024-01-24 00:00:00,2024-03-26 00:00:00,NaT,None,None,DOMÈSTIC,0801902046,45TBDJQN4LA37ZIN,NaN,2024-01-01
3,EU6AT3IKPUKCZTBU,2023-11-27 00:00:00,2024-01-24 00:00:00,NaT,None,None,DOMÈSTIC,0801902046,45TBDJQN4LA37ZIN,NaN,2024-01-01
4,EWNDTPECBVEGW6AU,2023-09-29 00:00:00,2023-11-27 00:00:00,NaT,None,None,DOMÈSTIC,0801902046,VTRAI3L24SWKVC5H,NaN,2024-01-01


In [4]:
# Display basic information about the DataFrame
print('DataFrame Information:')
print(df_ab3.info())

# Print number of null values in each column
print('\nNumber of null values in each column:')
print(df_ab3.isnull().sum())

DataFrame Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76372248 entries, 0 to 76372247
Data columns (total 11 columns):
 #   Column               Dtype         
---  ------               -----         
 0   POLISSA_SUBM         object        
 1   DATA_INI_FACT        object        
 2   DATA_FIN_FACT        object        
 3   CREATED_MENSAJE      datetime64[us]
 4   CODIGO_MENSAJE       object        
 5   TIPO_MENSAJE         object        
 6   US_AIGUA_SUBM        object        
 7   SECCIO_CENSAL        object        
 8   NUMEROSERIECONTADOR  object        
 9   CONSUMO_REAL         float64       
 10  FECHA_HORA           datetime64[us]
dtypes: datetime64[us](2), float64(1), object(8)
memory usage: 6.3+ GB
None

Number of null values in each column:
POLISSA_SUBM                  0
DATA_INI_FACT                 0
DATA_FIN_FACT                 0
CREATED_MENSAJE        23318973
CODIGO_MENSAJE         23318973
TIPO_MENSAJE           23318973
US_AIGUA_SUBM         

## 1.2. Tratamiento de los valores nulos

### 1.2.1. Toma de decisiones sobre los valores nulos en SECCIO_CENSAL

In [ ]:
# Check how many unique POLISSA_SUBM has missing values of SECCIO_CENSAL
missing_seccio_censal = df_ab3[df_ab3['SECCIO_CENSAL'].isnull()]['POLISSA_SUBM'].nunique()
print(f'Number of unique POLISSA_SUBM with missing SECCIO_CENSAL: {missing_seccio_censal}')

# Count total unique POLISSA_SUBM
total_unique_polisas = df_ab3['POLISSA_SUBM'].nunique()
print(f'\nTotal unique POLISSA_SUBM: {total_unique_polisas}')

# Check if there is a POLISSA_SUBM with some rows having SECCIO_CENSAL and others not
polisas_with_partial_seccio = df_ab3.groupby('POLISSA_SUBM')['SECCIO_CENSAL'].apply(lambda x: x.isnull().any() and x.notnull().any())
polisas_with_partial_seccio = polisas_with_partial_seccio[polisas_with_partial_seccio].index.tolist()
print(f'\nPOLISSA_SUBM with some rows having SECCIO_CENSAL and others not: {polisas_with_partial_seccio}')



Number of unique POLISSA_SUBM with missing SECCIO_CENSAL: 26
Total unique POLISSA_SUBM: 2697


Debido a que tenemos un número reducido de clientes (26 de 2697) de los cuales no conocemos su secció censal, se ha decidido eliminar todas las filas asociadas a estos clientes, ya que dicha información será esencial para nuestro modelo al realizar merge con datos abiertos.

In [8]:
# Drop rows with null SECCIO_CENSAL
df_ab3 = df_ab3.dropna(subset=['SECCIO_CENSAL'])

### 1.2.2. Imputación en valores nulos de la variable "CONSUMO_REAL"
Dada la información proporcionada por el equipo de AB Data, cuando la variable "CONSUMO_REAL" tiene un valor NaN quiere decir que el valor registrado es un valor < 1, para el correcto funcionamiento de los modelos a desarrollar asumiremos que este valor NaN será "0".

In [9]:
# Imputation of missing values in "CONSUMO_REAL" column
df_ab3.fillna({'CONSUMO_REAL': 0}, inplace=True)

### 1.2.3. Toma de decisiones sobre las variables "CREATED_MENSAJE", "CODIGO_MENSAJE" y "TIPO_MENSAJE"

Estas 3 variables hacen referencia a la detección y comunicación de fuga ('FUITA'), así como a su reiteración ('REITERACIÓ DE FUITA). Para llevar a cabo nuestro modelo predictivo nos hará falta simplificar el proceso de detección de fuga por lo que se ha decidido retirar estas columnas y añadir una nueva columna binaria que indica 0 si no hay fuga y 1 si hay fuga.

In [10]:
# Create binary column for leak detection
df_ab3['FUGA_DETECTADA'] = df_ab3['CODIGO_MENSAJE'].apply(lambda x: 1 if x in ['FUITA', 'REITERACIÓ DE FUITA'] else 0)

# Create binary column for reiteration of leak
df_ab3['FUGA_REITERADA'] = df_ab3['CODIGO_MENSAJE'].apply(lambda x: 1 if x == 'REITERACIÓ DE FUITA' else 0)

# Drop unnecessary columns
df_ab3.drop(columns=['CREATED_MENSAJE', 'CODIGO_MENSAJE', 'TIPO_MENSAJE'], inplace=True)

### 1.2.4. Verificación de la imputación en valores nulos

Verificamos si hemos realizado una correcta imputación imprimiendo el número de valores nulos en cada columna.

In [11]:
# Print number of null values in each column
print('Number of null values in each column:')
print(df_ab3.isnull().sum())

Number of null values in each column:
POLISSA_SUBM           0
DATA_INI_FACT          0
DATA_FIN_FACT          0
US_AIGUA_SUBM          0
SECCIO_CENSAL          0
NUMEROSERIECONTADOR    0
CONSUMO_REAL           0
FECHA_HORA             0
FUGA_DETECTADA         0
FUGA_REITERADA         0
dtype: int64


## 1.3. Tratamiento de la variable "FECHA_HORA" 

Para una mayor claridad de nuestros datos se ha decidido separar la variable "FECHA_HORA" en dos variables distintas: "FECHA" y "HORA".


In [12]:
# Convert specified columns to datetime format
for col in ["FECHA_HORA"]:
    df_ab3[col] = pd.to_datetime(df_ab3[col], errors="coerce")

# Split 'FECHA_HORA' into separate date and time columns
df_ab3['FECHA'] = df_ab3['FECHA_HORA'].dt.date
df_ab3['HORA'] = df_ab3['FECHA_HORA'].dt.time

# Cast 'FECHA' column to datetime
df_ab3['FECHA'] = pd.to_datetime(df_ab3['FECHA'], errors="coerce")

# Cast 'HORA' column to datetime
df_ab3['HORA'] = pd.to_datetime(df_ab3['HORA'].astype(str), format='%H:%M:%S', errors='coerce').dt.time

# Drop the original 'FECHA_HORA' column
df_ab3 = df_ab3.drop(columns=['FECHA_HORA'])

## 1.4. Ordenar las columnas del dataset

In [13]:
# Order columns
column_order = ['POLISSA_SUBM', 'NUMEROSERIECONTADOR', 'FECHA', 'HORA', 'SECCIO_CENSAL', 'US_AIGUA_SUBM', 'CONSUMO_REAL', 'FUGA_REITERADA', 'FUGA_DETECTADA']
df_ab3 = df_ab3[column_order]

# Display updated DataFrame 
print('Updated DataFrame:')
display(df_ab3.head())

Updated DataFrame:


,POLISSA_SUBM,NUMEROSERIECONTADOR,FECHA,HORA,SECCIO_CENSAL,US_AIGUA_SUBM,CONSUMO_REAL,FUGA_REITERADA,FUGA_DETECTADA
0,RGYFWIZ4ZRRZKX2K,IBAJ44VHSIRRTASA,2024-01-01,00:00:00,0801907090,DOMÈSTIC,0.0,0,0
1,HHB4U5HUQKW7IOGD,L2CLPPJRIPAEESV7,2024-01-01,00:00:00,0801909040,DOMÈSTIC,0.0,0,0
2,EU6AT3IKPUKCZTBU,45TBDJQN4LA37ZIN,2024-01-01,00:00:00,0801902046,DOMÈSTIC,0.0,0,0
3,EU6AT3IKPUKCZTBU,45TBDJQN4LA37ZIN,2024-01-01,00:00:00,0801902046,DOMÈSTIC,0.0,0,0
4,EWNDTPECBVEGW6AU,VTRAI3L24SWKVC5H,2024-01-01,00:00:00,0801902046,DOMÈSTIC,0.0,0,0


# 2. Open Data

## 2.1. Datos metereológicos

In [14]:
# Load meteorological data
df_aemet1 = pd.read_json('../data/open-data/data_aemet_1.json')

df_aemet2 = pd.read_json('../data/open-data/data_aemet_2.json')


# Merge meteorological datasets
df_aemet = pd.concat([df_aemet1, df_aemet2], ignore_index=True)

print('Meteorological DataFrame:')
display(df_aemet)

Meteorological DataFrame:


,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,horatmax,dir,velmedia,racha,horaracha,hrMedia,hrMax,horaHrMax,hrMin,horaHrMin
0,2024-01-01,0201D,"BARCELONA, PORT OLÍMPIC",BARCELONA,26,"11,6","0,0","8,6",06:00,"14,6",13:40,36.0,"1,9","4,2",06:20,82,90,16:50,69,12:30
1,2024-01-02,0201D,"BARCELONA, PORT OLÍMPIC",BARCELONA,26,"11,8","0,0","8,1",01:50,"15,4",12:20,26.0,"2,8","7,5",11:20,53,81,00:30,39,16:10
2,2024-01-03,0201D,"BARCELONA, PORT OLÍMPIC",BARCELONA,26,"16,0","0,0","13,6",00:30,"18,4",12:30,27.0,"1,4","9,7",11:50,74,86,19:50,62,00:00
3,2024-01-04,0201D,"BARCELONA, PORT OLÍMPIC",BARCELONA,26,"14,2","0,3","11,0",07:00,"17,4",13:30,26.0,"1,4","4,7",19:00,89,94,18:10,79,00:00
4,2024-01-05,0201D,"BARCELONA, PORT OLÍMPIC",BARCELONA,26,"11,2","8,4","8,9",21:50,"13,5",13:50,30.0,"2,2","6,9",23:30,89,94,Varias,78,13:30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,2024-12-27,0201D,"BARCELONA, PORT OLÍMPIC",BARCELONA,26,"12,5","0,0","7,9",23:59,"17,1",12:20,2.0,"1,7","6,4",04:50,62,95,Varias,39,14:20
362,2024-12-28,0201D,"BARCELONA, PORT OLÍMPIC",BARCELONA,26,"9,7","0,0","4,6",07:30,"14,8",13:30,20.0,"1,9","5,6",14:30,73,88,20:50,59,12:40
363,2024-12-29,0201D,"BARCELONA, PORT OLÍMPIC",BARCELONA,26,"10,6","0,0","5,1",07:40,"16,2",13:00,99.0,"2,5","5,0",Varias,69,86,03:40,46,12:40
364,2024-12-30,0201D,"BARCELONA, PORT OLÍMPIC",BARCELONA,26,"9,3","0,0","4,4",07:40,"14,2",13:00,21.0,"2,2","5,6",14:00,76,89,20:50,59,11:40


### 2.1.1. Comprobación de valores nulos del dataset

In [15]:
# Print number of null values in each column
print('Number of null values in each column:')
print(df_aemet.isnull().sum())

Number of null values in each column:
fecha         0
indicativo    0
nombre        0
provincia     0
altitud       0
tmed          0
prec          1
tmin          0
horatmin      1
tmax          0
horatmax      0
dir           1
velmedia      0
racha         1
horaracha     1
hrMedia       0
hrMax         0
horaHrMax     0
hrMin         0
horaHrMin     0
dtype: int64


Observamos que es un buen dataset por el poco número de valores nulos así que podemos utilizarlo de forma efectiva para nuestro dataset.


### 2.1.2. Selección de características relevantes para nuestro dataset principal

Analizaremos todas las variables medidas en cada estación de medición de nuestro dataset para seleccionar las más relevantes para nuestro objetivo.

**Variables Escogidas**
Nos centramos en filtrar el "ruido" (falsos positivos) causado por el clima cálido de Barcelona y en capturar causas directas de rotura.

- fecha: IMPRESCINDIBLE. Es la clave para unir (merge) estos datos climáticos con nuestro dataset de consumo (AB3).

- tmin: MUY IMPORTANTE. Es la temperatura mínima diaria. La usaremos para calcular TEMP_MIN_BCN para detectar heladas.

- tmax: MUY IMPORTANTE. Es la temperatura máxima diaria. La usaremos para calcular TEMP_MAX_BCN para filtrar el "ruido" por calor.

- prec: IMPORTANTE. Es la precipitación acumulada diaria. La usaremos para calcular PRECIP_MEAN_BCN como el "anulador del ruido" por calor.

- hrMedia: ÚTIL. Es la humedad relativa media diaria. La podemos usar (calculando la mean() entre estaciones) para afinar el contexto del calor (calor seco vs. calor húmedo).

In [16]:
# Select relevant features for our main dataset
relevant_features = ['fecha', 'tmed', 'tmin', 'tmax', 'prec', 'hrMedia']
df_aemet = df_aemet[relevant_features]
print('Selected relevant features for meteorological dataset:')
display(df_aemet.head())

Selected relevant features for meteorological dataset:


,fecha,tmed,tmin,tmax,prec,hrMedia
0,2024-01-01,"11,6","8,6","14,6","0,0",82
1,2024-01-02,"11,8","8,1","15,4","0,0",53
2,2024-01-03,"16,0","13,6","18,4","0,0",74
3,2024-01-04,"14,2","11,0","17,4","0,3",89
4,2024-01-05,"11,2","8,9","13,5","8,4",89


### 2.1.3. Imputación en valores nulos en la variable "prec"

In [17]:
print('Number of null values in each column after feature selection:')
print(df_aemet.isnull().sum())

# Imputation in null value of prec column with 0
df_aemet.fillna({'prec': 0}, inplace=True)

print('\nNumber of null values in each column after imputation:')
print(df_aemet.isnull().sum())

# Print number of null values
print('\nRows with null values:')
print(df_aemet.isnull().sum())


Number of null values in each column after feature selection:
fecha      0
tmed       0
tmin       0
tmax       0
prec       1
hrMedia    0
dtype: int64

Number of null values in each column after imputation:
fecha      0
tmed       0
tmin       0
tmax       0
prec       0
hrMedia    0
dtype: int64

Rows with null values:
fecha      0
tmed       0
tmin       0
tmax       0
prec       0
hrMedia    0
dtype: int64


### 2.1.4. Merge con el dataset principal

In [18]:
# Rename columns for clarity
cols_to_rename = {
    'fecha': 'FECHA',
    'tmed': 'TEMP_MEDIA',
    'tmin': 'TEMP_MIN',
    'tmax': 'TEMP_MAX',
    'prec': 'PRECIPITACION',
    'hrMedia': 'HUMEDAD_RELATIVA_MEDIA'
}
df_aemet.rename(columns=cols_to_rename, inplace=True)

# Cast FECHA columns to datetime
df_aemet['FECHA'] = pd.to_datetime(df_aemet['FECHA'], errors='coerce')

# Merge with main dataset
df_ab3_aemet = pd.merge(df_ab3, df_aemet, on='FECHA', how='left')

In [19]:
# Order columns
column_order_final = ['POLISSA_SUBM', 'NUMEROSERIECONTADOR', 'FECHA', 'HORA', 'SECCIO_CENSAL', 'US_AIGUA_SUBM', 'TEMP_MEDIA', 'TEMP_MIN', 'TEMP_MAX', 'PRECIPITACION', 'HUMEDAD_RELATIVA_MEDIA','CONSUMO_REAL','FUGA_REITERADA', 'FUGA_DETECTADA']
df_ab3_aemet = df_ab3_aemet[column_order_final]


print('Final merged DataFrame:')
display(df_ab3_aemet.head())

Final merged DataFrame:


,POLISSA_SUBM,NUMEROSERIECONTADOR,FECHA,HORA,SECCIO_CENSAL,US_AIGUA_SUBM,TEMP_MEDIA,TEMP_MIN,TEMP_MAX,PRECIPITACION,HUMEDAD_RELATIVA_MEDIA,CONSUMO_REAL,FUGA_REITERADA,FUGA_DETECTADA
0,RGYFWIZ4ZRRZKX2K,IBAJ44VHSIRRTASA,2024-01-01,00:00:00,0801907090,DOMÈSTIC,"11,6","8,6","14,6","0,0",82,0.0,0,0
1,HHB4U5HUQKW7IOGD,L2CLPPJRIPAEESV7,2024-01-01,00:00:00,0801909040,DOMÈSTIC,"11,6","8,6","14,6","0,0",82,0.0,0,0
2,EU6AT3IKPUKCZTBU,45TBDJQN4LA37ZIN,2024-01-01,00:00:00,0801902046,DOMÈSTIC,"11,6","8,6","14,6","0,0",82,0.0,0,0
3,EU6AT3IKPUKCZTBU,45TBDJQN4LA37ZIN,2024-01-01,00:00:00,0801902046,DOMÈSTIC,"11,6","8,6","14,6","0,0",82,0.0,0,0
4,EWNDTPECBVEGW6AU,VTRAI3L24SWKVC5H,2024-01-01,00:00:00,0801902046,DOMÈSTIC,"11,6","8,6","14,6","0,0",82,0.0,0,0


## 2.2. Gestión de días festivos

A continuación crearemos una variable booleana a través de la librería holidays para almacenar los días festivos puesto que este hecho se relacionará directamente con el consumo de agua.

In [20]:
# %pip install holidays
import holidays

### 2.2.1. Selección de días festivos y findes de semana

In [21]:
# Create a range of dates
fechas = pd.date_range(start='2024-01-01', end='2024-12-31')
df_fechas = pd.DataFrame({'FECHA': fechas})

# Get holidays for Spain, subdivision Catalonia (ES, CT)
es_holidays = holidays.CountryHoliday('ES', subdiv='CT', years=2024)

# Create the 'FESTIVO' column
df_fechas['FESTIVO'] = df_fechas['FECHA'].apply(lambda date: date in es_holidays)

### 2.2.2. Tener en cuenta días no laborales cotidianos

In [22]:
def get_tipo_dia_simple(fecha, es_festivo):
    if es_festivo: 
        return 'Festivo'
    elif fecha.weekday() >= 5: 
        return 'Fin de Semana'
    else:
        return 'Laborable'

df_fechas['TIPO_DIA'] = df_fechas.apply(lambda row: get_tipo_dia_simple(row['FECHA'], row['FESTIVO']), axis=1)

### 2.2.2. Merge con el dataset principal

In [23]:
# Cast FECHA column to datetime
df_fechas['FECHA'] = pd.to_datetime(df_fechas['FECHA'], errors='coerce')

# Merge holiday information with final dataset
df_ab3_aemet_fechas = pd.merge(df_ab3_aemet, df_fechas[['FECHA', 'FESTIVO', 'TIPO_DIA']], on='FECHA', how='right')

# Order columns 
column_order_final = ['POLISSA_SUBM', 'NUMEROSERIECONTADOR', 'FECHA', 'HORA', 'FESTIVO', 'TIPO_DIA', 'TEMP_MEDIA', 'TEMP_MIN', 'TEMP_MAX', 'PRECIPITACION', 'HUMEDAD_RELATIVA_MEDIA', 'CONSUMO_REAL', 'FUGA_DETECTADA']
df_ab3_aemet_fechas = df_ab3_aemet_fechas[column_order_final]

df_ab3_aemet_fechas.head()


,POLISSA_SUBM,NUMEROSERIECONTADOR,FECHA,HORA,FESTIVO,TIPO_DIA,TEMP_MEDIA,TEMP_MIN,TEMP_MAX,PRECIPITACION,HUMEDAD_RELATIVA_MEDIA,CONSUMO_REAL,FUGA_DETECTADA
0,RGYFWIZ4ZRRZKX2K,IBAJ44VHSIRRTASA,2024-01-01,00:00:00,True,Festivo,"11,6","8,6","14,6","0,0",82,0.0,0
1,HHB4U5HUQKW7IOGD,L2CLPPJRIPAEESV7,2024-01-01,00:00:00,True,Festivo,"11,6","8,6","14,6","0,0",82,0.0,0
2,EU6AT3IKPUKCZTBU,45TBDJQN4LA37ZIN,2024-01-01,00:00:00,True,Festivo,"11,6","8,6","14,6","0,0",82,0.0,0
3,EU6AT3IKPUKCZTBU,45TBDJQN4LA37ZIN,2024-01-01,00:00:00,True,Festivo,"11,6","8,6","14,6","0,0",82,0.0,0
4,EWNDTPECBVEGW6AU,VTRAI3L24SWKVC5H,2024-01-01,00:00:00,True,Festivo,"11,6","8,6","14,6","0,0",82,0.0,0


## 2.3. Datos geográficos

In [ ]:
import pandas as pd
import dask.dataframe as dd
import numpy as np
import holidays
import gc

# --- 0. Configuración General ---
MAIN_PARQUET = '../data/official-data/data_ab3_complete.parquet'
OUTPUT_PARQUET_DIR = 'dataset_FINAL_COMPLETO/'
NUMERO_DE_TROZOS = 60
LLAVE_DISTRITO = 'KEY_DISTRITO'
LLAVE_SECCION = 'KEY_SECCION'


RENTA_CSV = '../data/open-data/renda_procesada.csv'
ANTIGUITAD_CSV = '../data/open-data/antiguitat_pivotada.csv'
POBLACION_CSV = '../data/open-data/poblacion_pivotada.csv'
OBRES_CSV = '../data/open-data/obres_procesadas.csv'
# --- FIN DE LA ACTUALIZACIÓN ---

# --- 1. Cargar y Preparar Tablas Pequeñas (con PANDAS) ---
print("Cargando 4 datasets geográficos...")
dtype_llaves = {LLAVE_DISTRITO: 'string', LLAVE_SECCION: 'string'}
df_renta = pd.read_csv(RENTA_CSV, dtype=dtype_llaves)
df_antiguitat = pd.read_csv(ANTIGUITAD_CSV, dtype=dtype_llaves)
df_poblacion = pd.read_csv(POBLACION_CSV, dtype=dtype_llaves)
df_obres = pd.read_csv(OBRES_CSV, dtype=dtype_llaves)

print("Uniendo archivos geográficos...")
df_geo_total = df_renta.merge(df_antiguitat, on=[LLAVE_DISTRITO, LLAVE_SECCION], how='outer')
df_geo_total = df_geo_total.merge(df_poblacion, on=[LLAVE_DISTRITO, LLAVE_SECCION], how='outer')
df_geo_total = df_geo_total.merge(df_obres, on=[LLAVE_DISTRITO, LLAVE_SECCION], how='outer')
nuevas_columnas_geo = df_geo_total.columns.drop([LLAVE_DISTRITO, LLAVE_SECCION]).tolist()
df_geo_total = df_geo_total.fillna(0)
print(f"Tabla de características GEO creada con {len(df_geo_total)} filas.")

# 1B: Preparar datos METEO (Del Notebook)
print("Cargando datos de AEMET...")
df_aemet1 = pd.read_json('../data/open-data/data_aemet_1.json')
df_aemet2 = pd.read_json('../data/open-data/data_aemet_2.json')
df_aemet = pd.concat([df_aemet1, df_aemet2], ignore_index=True)
df_aemet = df_aemet[['fecha', 'tmed', 'tmin', 'tmax', 'prec', 'hrMedia']]
df_aemet.fillna({'prec': 0}, inplace=True)
cols_to_rename = {
    'fecha': 'FECHA', 'tmed': 'TEMP_MEDIA', 'tmin': 'TEMP_MIN',
    'tmax': 'TEMP_MAX', 'prec': 'PRECIPITACION', 'hrMedia': 'HUMEDAD_RELATIVA_MEDIA'
}
df_aemet.rename(columns=cols_to_rename, inplace=True)
df_aemet['FECHA'] = pd.to_datetime(df_aemet['FECHA'], errors='coerce')
nuevas_columnas_meteo = list(cols_to_rename.values())[1:]

# 1C: Preparar datos FESTIVOS (Del Notebook)
print("Generando datos de Festivos...")
fechas = pd.date_range(start='2024-01-01', end='2024-12-31')
df_fechas = pd.DataFrame({'FECHA': fechas})
es_holidays = holidays.CountryHoliday('ES', subdiv='CT', years=2024)
df_fechas['FESTIVO'] = df_fechas['FECHA'].apply(lambda date: date in es_holidays)
def get_tipo_dia_simple(fecha, es_festivo):
    if es_festivo: return 'Festivo'
    elif fecha.weekday() >= 5: return 'Fin de Semana'
    else: return 'Laborable'
df_fechas['TIPO_DIA'] = df_fechas.apply(lambda row: get_tipo_dia_simple(row['FECHA'], row['FESTIVO']), axis=1)
df_fechas['FECHA'] = pd.to_datetime(df_fechas['FECHA'], errors='coerce')
nuevas_columnas_festivos = ['FESTIVO', 'TIPO_DIA']

# 1D: Unir tablas pequeñas de Meteo y Festivos
df_meteo_festivos = pd.merge(df_aemet, df_fechas, on='FECHA', how='outer')

# --- 2. Cargar y Preparar Tabla Gigante (con DASK) ---
print("Cargando archivo principal (76M filas) con DASK...")
df_main_dask = dd.read_parquet(MAIN_PARQUET)
print(f"Creando {NUMERO_DE_TROZOS} trozos más pequeños...")
df_main_dask = df_main_dask.repartition(npartitions=NUMERO_DE_TROZOS)

# 2A: Limpiar SECCIO_CENSAL
df_main_dask = df_main_dask.dropna(subset=['SECCIO_CENSAL'])

# 2B: Imputar CONSUMO_REAL
df_main_dask['CONSUMO_REAL'] = df_main_dask['CONSUMO_REAL'].fillna(0)

# 2C: Crear Columnas FUGA (Manejando nulos)
def crear_columnas_fuga(df_particion):
    df_particion['FUGA_DETECTADA'] = df_particion['CODIGO_MENSAJE'].apply(
        lambda x: 0 if pd.isna(x) else (1 if x in ['FUITA', 'REITERACIÓ DE FUITA'] else 0)
    )
    df_particion['FUGA_REITERADA'] = df_particion['CODIGO_MENSAJE'].apply(
        lambda x: 0 if pd.isna(x) else (1 if x == 'REITERACIÓ DE FUITA' else 0)
    )
    return df_particion

print("Creando columnas de Fuga...")
new_meta = df_main_dask._meta.copy()
new_meta['FUGA_DETECTADA'] = pd.Series(dtype='int64')
new_meta['FUGA_REITERADA'] = pd.Series(dtype='int64')
df_main_dask = df_main_dask.map_partitions(crear_columnas_fuga, meta=new_meta)
df_main_dask = df_main_dask.drop(columns=['CREATED_MENSAJE', 'CODIGO_MENSAJE', 'TIPO_MENSAJE'])

# 2D: Separar FECHA_HORA
print("Separando Fecha y Hora...")
df_main_dask['FECHA'] = df_main_dask['FECHA_HORA'].dt.date
df_main_dask['HORA'] = df_main_dask['FECHA_HORA'].dt.time.astype(str) # Corrección V12
df_main_dask['FECHA'] = dd.to_datetime(df_main_dask['FECHA'], errors='coerce')

# --- 3. MERGES FINALES (con DASK) ---
print("Preparando Merge 1 (Meteo + Festivos)...")
df_main_dask = dd.merge(
    df_main_dask,
    df_meteo_festivos,
    on='FECHA',
    how='left'
)

print("Preparando Merge 2 (Geográfico)...")
llave_str = df_main_dask['SECCIO_CENSAL'].astype('string')
df_main_dask[LLAVE_DISTRITO] = llave_str.str.slice(5, 7).astype('string')
df_main_dask[LLAVE_SECCION] = llave_str.str.slice(7, 10).astype('string')

df_final_dask = dd.merge(
    df_main_dask,
    df_geo_total,
    on=[LLAVE_DISTRITO, LLAVE_SECCION],
    how='left'
)

# --- 4. Limpieza y Guardado Final ---
del df_renta, df_antiguitat, df_poblacion, df_obres, df_geo_total, df_aemet, df_fechas, df_meteo_festivos
gc.collect()

print("Limpiando nulos finales...")
fill_values_geo = {col: 0 for col in nuevas_columnas_geo}
fill_values_meteo = {col: 0 for col in nuevas_columnas_meteo}
fill_values_festivos = {'FESTIVO': False, 'TIPO_DIA': 'Laborable'} 

df_final_dask = df_final_dask.fillna(value=fill_values_geo)
df_final_dask = df_final_dask.fillna(value=fill_values_meteo)
df_final_dask = df_final_dask.fillna(value=fill_values_festivos)
df_final_dask['FESTIVO'] = df_final_dask['FESTIVO'].astype(bool)

# --- 5. Ejecutar y Guardado ---
print("¡EJECUTANDO! Dask está procesando TODO y guardando en disco...")
df_final_dask.to_parquet(OUTPUT_PARQUET_DIR, write_index=False)

print(f"¡HECHO! Tu dataset final y completo está en la CARPETA: '{OUTPUT_PARQUET_DIR}'")